In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 Document AI 和 PaLM API 摘要大型文件

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/doggy8088/generative-ai/blob/main/language/use-cases/document-summarization/summarization_with_documentai.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory 圖標"><br> 在 Colab 中執行
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/doggy8088/generative-ai/blob/main/language/use-cases/document-summarization/summarization_with_documentai.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub 圖標"><br> 在 GitHub 上查看
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/doggy8088/generative-ai/blob/main/language/use-cases/document-summarization/summarization_with_documentai.zh.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI 圖標"><br> 在 Vertex AI Workbench 中開啟
    </a>
  </td>
</table>


| | |
|-|-|
|作者| [Holt Skinner](https://github.com/holtskinner), [Mona Mona](https://github.com/Mona19) |


## 概述

文字摘要是指在仍保留最重要訊息的情況下，建立文字文件較短版本的程序。這對於各種用途都很有用，例如快速略讀長文件、掌握文章大意或與他人分享摘要。

儘管摘要簡短段落是一項非瑣碎任務，但如果你想要摘要一份大型文件，例如包含多頁的 PDF 文件，那麼有幾個挑戰需要克服。

[Document AI](https://cloud.google.com/document-ai) 提供一種可擴充且受管理的方法，可以利用 AI 從文件中提取資料。在本筆記本中，你將使用 [Document OCR 處理器](https://cloud.google.com/document-ai/docs/document-ocr)，這是一個預先訓練好的模型，將從文件檔中提取文字和版面資訊。Document AI 提供一個 API 端點來存取這些模型，讓開發人員不必建置和維護自己的模型和服務基礎架構。


### 目標

在此筆記本中，我們將展示你如何執行以下工作：

1. 使用 Document AI OCR 處理器從 PDF 文件中擷取文字。
1. 使用 MapReduce 方法對文件文字進行分段。
1. 使用 PaLM `text-bison@001` 模型為擷取的文字產生摘要。


### 成本

本教學使用 Google Cloud 的計費元件：

- Generative AI 工作室提供的 Vertex AI PaLM API
- Document AI

了解 [Vertex AI 定價](https://cloud.google.com/vertex-ai/pricing)，
了解 [Document AI 定價](https://cloud.google.com/document-ai/pricing)，
並使用 [定價計算器](https://cloud.google.com/products/calculator/)
根據你的預測使用量來估算成本。


## 開始使用


### 安裝 Vertex AI SDK 和其他相依性


In [ ]:
%pip install --upgrade google-cloud-aiplatform==1.35.0 google-cloud-documentai==2.20.1 google-cloud-documentai-toolbox==0.11.1a backoff==2.2.1 --user

**Colab 只限** ：執行以下Cell以重新啟動核心。對於 Vertex AI Workbench，你可以使用最上方的按鈕重新啟動終端機。


In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### 在專案中啟用 APIs


In [ ]:
!gcloud config set project "YOUR_PROJECT_ID"
!gcloud services enable documentai.googleapis.com storage.googleapis.com aiplatform.googleapis.com

## Document AI

以下 [限制](https://cloud.google.com/document-ai/quotas) 適用於 Document OCR 處理器的線上處理。

| 限制                      | 值 |
| :------------------------ | ----: |
| 最大檔案大小           | 20 MB |
| 最大頁數               |    15 |

對於不符合這些限制的文件，你可以使用 [批次處理](https://cloud.google.com/document-ai/docs/send-request#batch-process) 提取文件文字。(未在本筆記中作介紹。)


### 準備資料檔

首先，你需要下載以下摘要任務的 PDF。
使用此筆記本電腦時，你將使用儲存在公開的 Google Cloud Storage 儲存空間中的 Alphabet 收益報告 PDF。


In [ ]:
# Copying the files from the GCS bucket to local storage
!gsutil -m cp -r gs://github-repo/documents/docai .

### 建立 Document AI OCR 處理器

[Document AI 處理器](https://cloud.google.com/document-ai/docs/overview#dai-processors) 在文件與執行文件處理動作的機器學習模型之間形成介面。這些處理器可為文件進行分類、拆分、語法剖析或分析。每個 Google Cloud 專案都需要建立自己的處理器執行個體。

Document AI 處理器有兩個類型：

- 預先訓練的處理器：這些處理器預先在大量文件資料集上進行訓練，可用於執行常見的文件處理工作，例如光學字元辨識 (OCR)、表單語法剖析，和實體萃取。
- 自訂處理器：這些處理器可在你自己的文件資料集上進行訓練，藉以執行預先訓練的處理器未涵蓋的特定工作。

請參閱 [完整處理器及詳細清單](https://cloud.google.com/document-ai/docs/processors-list) 以取得所有支援的處理器。

處理器會將 PDF 或影像檔案作為輸入，並以 [`Document`](https://cloud.google.com/document-ai/docs/reference/rest/v1/Document) 格式輸出資料。


### 建立處理器

僅執行此程式碼一次以建立處理器。你無法使用相同的顯示名稱建立多個處理器。如果你收到錯誤訊息，請變更處理器名稱並重新執行。


In [ ]:
from google.api_core.client_options import ClientOptions
from google.api_core.exceptions import AlreadyExists
from google.cloud import documentai
from google.cloud.documentai_toolbox.wrappers.document import Document

# TODO(developer): Edit these variables before running the code.
project_id = "YOUR_PROJECT_ID"

# See https://cloud.google.com/document-ai/docs/regions for all options.
location = "us"

# Must be unique per project, e.g.: "My Processor"
processor_display_name = "YOUR_PROCESSOR_DISPLAY_NAME"

# You must set the `api_endpoint` if you use a location other than "us".
client_options = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")


def create_processor(
    project_id: str, location: str, processor_display_name: str
) -> documentai.Processor:
    client = documentai.DocumentProcessorServiceClient(client_options=client_options)

    # The full resource name of the location
    # e.g.: projects/project_id/locations/location
    parent = client.common_location_path(project_id, location)

    # Create a processor
    return client.create_processor(
        parent=parent,
        processor=documentai.Processor(
            display_name=processor_display_name, type_="OCR_PROCESSOR"
        ),
    )


try:
    processor = create_processor(project_id, location, processor_display_name)
    print(f"Created Processor {processor.name}")
except AlreadyExists as e:
    print(
        f"Processor already exits, change the processor name and rerun this code. {e.message}"
    )

### 處理文件

處理文件會取得處理器名稱和文件的檔案路徑，並從文件中萃取文字。


In [ ]:
def process_document(
    processor_name: str,
    file_path: str,
) -> documentai.Document:
    client = documentai.DocumentProcessorServiceClient(client_options=client_options)

    # Read the file into memory
    with open(file_path, "rb") as image:
        image_content = image.read()

    # Load Binary Data into Document AI RawDocument Object
    raw_document = documentai.RawDocument(
        content=image_content, mime_type="application/pdf"
    )

    # Configure the process request
    request = documentai.ProcessRequest(name=processor_name, raw_document=raw_document)

    result = client.process_document(request=request)

    return result.document

#### 建立資料區塊

若於加入提示之前，將輸入資料分割成小「區塊」，LLM 在文件摘要處理上能產出最佳結果。

最佳區塊大小會取決於文件大小。建議使用不同區塊大小進行實驗，以了解對特定資料集和應用程式的最佳做法。

對於提供的文件，我們使用 Document AI 偵測的段落來區分區塊。

你也應該嘗試其他值，並了解其對摘要的影響。


In [ ]:
import glob
import os
from typing import Dict, List

# If you already have a Document AI Processor in your project, assign the full processor resource name here.
processor_name = processor.name
extracted_data: List[Dict] = []

# Loop through each PDF file in the "docai" directory.
for path in glob.glob("docai/*.pdf"):
    # Extract the file name and type from the path.
    file_name, file_type = os.path.splitext(path)

    print(f"Processing {file_name}")

    # Process the document.
    document = process_document(processor_name, file_path=path)

    if not document:
        continue

    # Using Document AI Toolbox to handle post-processing
    wrapped_document = Document.from_documentai_document(document)

    # Split the text into chunks based on paragraphs.
    document_chunks = [
        paragraph.text
        for page in wrapped_document.pages
        for paragraph in page.paragraphs
    ]

    # Can also split into chunks by page or blocks.
    # document_chunks = [page.text for page in wrapped_document.pages]
    # document_chunks = [block.text for page in wrapped_document.pages for block in page.blocks]

    # Loop through each chunk and create a dictionary with metadata and content.
    for chunk_number, chunk_content in enumerate(document_chunks, start=1):
        # Append the chunk information to the extracted_data list.
        extracted_data.append(
            {
                "file_name": file_name,
                "file_type": file_type,
                "chunk_number": chunk_number,
                "content": chunk_content,
            }
        )

## 使用 [PaLM] (https://ai.google/discover/palm2/) 模型進行摘要

你剛剛使用 Document AI 從 PDF 檔案中萃取文字。

在下一節中，你將使用 Vertex AI 中的 PaLM 模型對萃取的文字進行摘要。
為了對文字進行摘要，你可以使用 MapReduce 將文字分塊以符合提示大小。


### 驗證你的筆記本環境

-如果你使用 **Colab** 執行此筆記本，執行下列Cell並繼續。
-如果你使用 **Vertex AI Workbench** ，請查看 [這裡](https://github.com/doggy8088/generative-ai/tree/main/setup-env) 的設定說明。


In [ ]:
import vertexai
from google.colab import auth

auth.authenticate_user()

PROJECT_ID = "YOUR_PROJECT_ID"  # @param {type:"string"}
vertexai.init(project=PROJECT_ID, location="us-central1")

### 匯入模型


In [ ]:
from __future__ import annotations
import backoff

from google.api_core.exceptions import ResourceExhausted

from vertexai.preview.language_models import TextGenerationModel

generation_model = TextGenerationModel.from_pretrained("text-bison@001")


# This decorator is used to handle exceptions and apply exponential backoff in case of ResourceExhausted errors.
# It means the function will be retried with increasing time intervals in case of this specific exception.
@backoff.on_exception(backoff.expo, ResourceExhausted, max_time=10)
def text_generation_model_with_backoff(**kwargs):
    """
    :param **kwargs: Keyword arguments for the prediction.
    :return: The generated text.
    """
    # Calls the generation_model's 'predict' method with the provided keyword arguments (**kwargs)
    # and then accesses the 'text' attribute to get the generated text.
    return generation_model.predict(**kwargs).text

## MapReduce

MapReduce 是一種非常有效的方法來處理大型資料集，原因在於它具備可擴充性和效率。它可以用於處理過大而無法在單一機器上處理的資料集。

使用這個方法時，我們會先將大量資料分割成區塊，然後在每個文字區塊上執行提示。對於摘要任務而言，初始提示的輸出將會是該區塊的摘要。在產生所有初始輸出後，再執行不同的提示來結合它們。這對於大型資料集來說會更有效率。

它包含兩個主要步驟，映射與歸約：

- 映射步驟會將資料集分割成區塊，並在每個文字區塊上執行提示。提示的輸出是該區塊的摘要。

- 歸約步驟會將所有區塊的摘要結合為一個摘要。

以下是使用 MapReduce 方法進行摘要的優缺點：

優點：

- 可以摘要大型文件
- 可以與平行處理搭配使用，因為摘要頁面的程序彼此獨立。

缺點：

- 需要多次呼叫模型
- 由於各個頁面都個別摘要，因此各頁面之間的文脈可能會遺失。


#### 對照步驟

在本部分，你將使用模型，透過初始提示範本，個別為每個段落文字摘要。


In [ ]:
prompt_template = """
    Write a concise summary of the following text delimited by triple backquotes.

    ```{text}```

    CONCISE SUMMARY:
"""

# Create an empty list to store the summaries
initial_summary: List[str] = []

# Iterate over the pages and generate a summary for each page
for individual_chunk in extracted_data:
    # Create a prompt for the model using the extracted text and a prompt template
    prompt = prompt_template.format(text=individual_chunk["content"].strip())

    # Generate a summary using the model and the prompt
    summary = text_generation_model_with_backoff(prompt=prompt, max_output_tokens=1024)

    # Append the summary to the list of summaries
    initial_summary.append(summary)

看看來自初始 Map 階段的前幾個摘要。這些摘要是每個個別文本段落的摘要。


In [ ]:
print("\n\n".join(initial_summary[:5]))

#### 縮減步驟

在這裡，你將建立一個縮減功能，它會串接來自初始摘要步驟 (對應步驟) 的摘要，並使用最終提示範本來建立初級摘要的摘要。


In [ ]:
# Concatenate the summaries from the inital step
concat_summary = "\n".join(initial_summary)

# Create a prompt for the model using the concatenated text and a prompt template
prompt = prompt_template.format(text=concat_summary)

# Generate a summary using the model and the prompt
summary = text_generation_model_with_backoff(prompt=prompt, max_output_tokens=34)
print(summary)

# 結論

在此筆記本中，你了解了：

1. 如何使用 Document AI 從這些 PDF 中提取文字。
2. 如何使用 MapReduce 有效地處理大量文字資料。
3. 如何使用 PaLM `text-bison@001` 模型來整理從 PDF 中提取出的文字。


## 清除垃圾

如果你不再需要 Document AI 處理器，可以使用以下程式碼將其刪除。

或者，你可以使用 Cloud Console 來刪除處理器，如 [建立並管理處理器 > 刪除處理器](https://cloud.google.com/document-ai/docs/create-processor#documentai_delete_processor-web) 中所述。


In [ ]:
def delete_processor(processor_name: str) -> None:
    client = documentai.DocumentProcessorServiceClient(client_options=client_options)

    # Delete a processor
    operation = client.delete_processor(name=processor_name)
    # Print operation details
    print(operation.operation.name)
    # Wait for operation to complete
    operation.result()


delete_processor(delete_processor, processor_name)